In [ ]:
from pathlib import Path

from toolbox_continu_inzicht.base.config import Config
from toolbox_continu_inzicht.base.data_adapter import DataAdapter

In [ ]:
path = Path.cwd() / "data_sets"
config = Config(config_path=path / "example_inspection.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
data_adapter.input("inspections_results")

Filter resultaten uit de input tabel:
```yaml
GlobalVariables:
    
    ...

    Filter:
        query: "prioriteit > 0"
        drop_columns:
            - "waterschap"
            - "location_id"
```

In [ ]:
from toolbox_continu_inzicht.inspections.filter import Filter

In [ ]:
filter = Filter(data_adapter=data_adapter)
filter.run(input="inspections_results", output="filter_resultaten")

In [ ]:
filter.df_out

De gefilterde tabel kunnen we vervolgens classificeren<br>
Als we geen opmaak informatie meegeven in de input wordt de standaard opmaak toegepast

In [ ]:
from toolbox_continu_inzicht.inspections.inspections import ClassifyInspections

In [ ]:
classify_inspections = ClassifyInspections(data_adapter=data_adapter)
classify_inspections.run(input="filter_resultaten", output="classify_resultaten")

In [ ]:
classify_inspections.df_out

Als we wel styling mee geven wordt dit toegepast

In [ ]:
data_adapter.input("styling_example")

In de config moeten we alleen de kolom opgeven waarop geclassificeerd wordt. 

```yaml
GlobalVariables:
    
    ...

    ClassifyInspections:
        classify_column: "prioriteit"
```

In [ ]:
classify_inspections_styling = ClassifyInspections(data_adapter=data_adapter)
classify_inspections_styling.run(
    input=["filter_resultaten", "styling_example"], output="classify_resultaten"
)

In [ ]:
classify_inspections_styling.df_out

Stel we willen naast `color` ook `fillColor` toevoegen en een Marker met symbool in plaats van rondje, dan zetten we deze kolom ook in de opmaak tabel

In [ ]:
data_adapter.input("more_styling_example")

In [ ]:
classify_inspections_more_styling = ClassifyInspections(data_adapter=data_adapter)
classify_inspections_more_styling.run(
    input=["filter_resultaten", "more_styling_example"], output="classify_resultaten"
)

En dan zien we dat fillColor nu ook geclassificeerd is:

In [ ]:
data_adapter.input("inspections_results")

In [ ]:
classify_inspections_more_styling.df_out

Dit is al een geo object wat we hier kunnen tonen

In [ ]:
import geopandas as gpd

In [ ]:
ax = classify_inspections_more_styling.df_out.plot()
gpd.read_file(path / "provincie_flevoland.geojson").plot(
    ax=ax, color="white", edgecolor="black", zorder=-1
)

Voor de viewer is een legenda tabel ook nuttig, deze kan als extra output worden opgegeven

In [ ]:
classify_inspections_more_styling = ClassifyInspections(data_adapter=data_adapter)
classify_inspections_more_styling.run(
    input=["filter_resultaten", "more_styling_example"],
    output=["classify_resultaten", "legenda"],
)

In [ ]:
classify_inspections_more_styling.df_legend_out

Hier gebruiken we een `lower_boundary` en `upper_boundary`, het werkt ook met alleen een `lower_boundary`

In [ ]:
data_adapter.input("styling_lower_bound")

In [ ]:
classify_inspections_lower_bound = ClassifyInspections(data_adapter=data_adapter)
classify_inspections_lower_bound.run(
    input=["filter_resultaten", "styling_lower_bound"],
    output=["classify_resultaten", "legenda"],
)
classify_inspections_lower_bound.df_out

Naast classificeren op numerieke waarde (prioriteit) kan het ook op text, in dit geval opmerking

In [ ]:
path = Path.cwd() / "data_sets"
config = Config(config_path=path / "example_inspection_text.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)
data_adapter.input("styling_example_text")

In [ ]:
classify_inspections_text = ClassifyInspections(data_adapter=data_adapter)
classify_inspections_text.run(
    input=["filter_resultaten", "styling_example_text"],
    output=["classify_resultaten", "legenda"],
)
classify_inspections_text.df_out

De output van de inspecties kan klaar gezet worden voor de database

In [ ]:
from toolbox_continu_inzicht.inspections.inspections import InspectionsToDatabase

In [ ]:
inspections_to_database = InspectionsToDatabase(data_adapter=data_adapter)
inspections_to_database.run(
    input=["classify_resultaten", "legenda"],
    output="example_to_database",
)

In [ ]:
inspections_to_database.df_out

Als we niks op geven in de als `layers` wordt er automatisch een aantal standaard opties overgenomen  

In [ ]:
inspections_to_database.df_in_layers

Deze kan ook uitgebreider

In [ ]:
data_adapter.input("layers")


```yaml
GlobalVariables:
    
    ...

    InspectionsToDatabase:
        index: 2
```

In [ ]:
inspections_to_database = InspectionsToDatabase(data_adapter=data_adapter)
inspections_to_database.run(
    input=["classify_resultaten", "legenda", "layers"],
    output="example_to_database",
)

In [ ]:
inspections_to_database.df_out

Tot nu toe was alles lokaal naar csv, maar dit kan juist naar de database

In [ ]:
path = Path.cwd() / "data_sets"
config = Config(config_path=path / "example_inspection_db.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
inspections_to_database.df_in_layers

In [ ]:
inspections_to_database = InspectionsToDatabase(data_adapter=data_adapter)
inspections_to_database.run(
    input=["classify_resultaten", "legenda", "layers"],
    output="to_ci_database",
)

In [ ]:
inspections_to_database.df_out